# Motivating example: Figure 4

In [42]:
import pandas as pd

from eliater.network_validation import conditional_independence_test_summary

# from y0.algorithm.taheri_design import simplify_latent_dag
# from IPython.display import Image, display
from src.eliater.frontdoor_backdoor_discrete import (
    single_mediator_with_multiple_confounders_nuisances_discrete_example,
)

# from y0.examples import identifiability_2
from y0.algorithm.identify import Identification, identify
from y0.dsl import X, Y
from y0.graph import NxMixedGraph

This is the motivating example in Figure 4 (a) in this paper: *Eliater: an analytical workflow and open source implementation for causal query
estimation in biomolecular networks*. 

In [43]:
graph = NxMixedGraph.from_str_edges(
    directed=[
        ("Z1", "X"),
        ("Z1", "Z2"),
        ("Z2", "Z3"),
        ("Z3", "Y"),
        ("X", "M1"),
        ("M1", "Y"),
        ("M1", "R1"),
        ("R1", "R2"),
        ("R2", "R3"),
        ("Y", "R3"),
    ],
    undirected=[
        # ("Z1", "X"),
    ],
)

In [44]:
data = single_mediator_with_multiple_confounders_nuisances_discrete_example.generate_data(500)

## Step 1: Verify correctness of the network structure

In [ ]:
#remove after branch is merged

In [44]:
conditional_independence_test_summary(graph, data, verbose=True)

## Step 2: Check query identifiability

## Step 3: Find nuisance variables and mark them as latent

## Step 4: Simplify the network

## Step 5: Estimate the query